In [ ]:
!pip install PyPDF2
!pip install transformer
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import spacy
import string
import re
import PyPDF2
import os

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1595: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language mo

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

In [ ]:
def remove_number_references(text):
    # Define a regular expression pattern to match the number references before [ ]
    pattern = r'\b\d+\['

    # Use re.sub to replace all matches with an empty string
    clean_text1 = re.sub(pattern, '[', text)
    clean_text2 = re.sub(r'\b\d+\.', '', clean_text1)
    clean_text3 = re.sub(r'\([^)]*\)', '', clean_text2)
    clean_text4 = re.sub(r'\b[A-Za-z]\.\b', '', clean_text3)
    # clean_text5 = re.sub(r'^\d+\s*', '', clean_text4)
    clean_text5 = re.sub(r'\s{2,}', ' ', clean_text4)

    return clean_text5

In [ ]:
def remove_puncts(input_string, string):

  if "\n                                                " in input_string:
    index = input_string.find('\n                                                ')
    input_string = input_string[:index]
  input = remove_number_references(input_string)
  input = input.replace("\n"," ")
  exclude = set(string.punctuation) - set('-') - set('.')
  translation_table = str.maketrans('', '', ''.join(exclude))
  input = input.translate(translation_table).lower()

  return input

In [ ]:
def get_answer(removed_punc_arr):
  sentences = {}
  for s_text in removed_punc_arr:
    doc = nlp(s_text)
    for ent in doc.ents:
      if ent.label_ == "DATE" or ent.label_ == "ORG" or ent.label_ == "LAW":
        if s_text != ent.text:
          if s_text in sentences and ent.text not in sentences[s_text]:
              sentences[s_text].append(ent.text)
          else:
            sentences[s_text] = [ent.text]

  return sentences

In [ ]:
def get_ans_ind(sentences):
  indexes = {}
  for key in sentences:
    for val in sentences[key]:
      word_ind = key.find(val)
      indexes[key+val] = word_ind

  return indexes

In [ ]:
def get_question(answer, context, max_length=1024):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'],
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return tokenizer.decode(output[0])

In [ ]:
def gen_que(train_file,sentences,indexes):
  id = 1
  for sen in sentences:
    for ans in sentences[sen]:
      answer = {
                "text": ans,
                "answer_start": indexes[sen+ans],
                "answer_end": len(str(ans)) + indexes[sen+ans]
            }
      que = get_question(ans, sen)
      start_ind = que.find("question")
      question = que[start_ind+10:-4]
      train_file.append({
          "context": sen,
          "question": question,
          "answer": answer,
          "id":id
      })
      id += 1

  return train_file

In [ ]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
      reader = PyPDF2.PdfReader(file)
      pdf_text = ''
      for page_num in range(1, len(reader.pages)):
        page = reader.pages[page_num]

        # extracting text from page
        text = page.extract_text()
        remove_text = remove_puncts(text,string)
        pdf_text += remove_text
    return pdf_text

In [ ]:
def find_title(pdf_path):
  if pdf_path == '/content/drive/MyDrive/acts/A1860-45.pdf':
    return 'THE INDIAN PENA'
  with open(pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    input_string = reader.pages[0].extract_text()
  start_ind = input_string.find("THE")
  title_ind = input_string.find(",")
  title = input_string[start_ind:title_ind]

  return title

In [ ]:
# with open('/content/drive/MyDrive/acts/196234.pdf', 'rb') as file:
#   reader = PyPDF2.PdfReader(file)
#   page = reader.pages[0]
#   print(page.extract_text())

In [ ]:
# with open('/content/drive/MyDrive/acts/A1860-45.pdf', 'rb') as file:
#   reader = PyPDF2.PdfReader(file)
#   page = reader.pages[0]
#   text = page.extract_text()
# print(text)

In [ ]:
# Directory containing PDF files
path = '/content/drive/MyDrive/acts'

# List PDF files in the directory
pdf_files = [os.path.join(path, file) for file in os.listdir(path)]

titles = []
# Extract text from each PDF file
documents = []
for pdf_file in pdf_files:
    text = extract_text_from_pdf(pdf_file)
    documents.append(text)
    titles.append(find_title(pdf_file))

titles

['THE INDIAN PENA',
 'THE PROTECTION OF CHILDREN FROM SEXUAL OFFENCES ACT',
 'THE CODE OF CRIMINAL PROCEDURE (AMENDMENT) ACT',
 'THE INDIAN EVIDENCE ACT',
 'THE ARMS  ACT',
 'THE INFORMATION TECHNOLOGY ACT',
 'THE NARCOTIC DRUGS AND PSYCHOTROPIC SUBSTANCES',
 'THE CONTEMPT OF COURTS ACT',
 'THE UNLAWFUL ACTIVITIES (PREVENTION)  ACT',
 'THE CRIMINAL PROCEDURE (IDENTIFICATION) ACT',
 'THE PREVENTION OF MONEY -LAUNDERING ACT',
 'THE EXTRADITION ACT',
 'THE PRISONS ACT',
 'THE PREVENTION OF CORRUPTION ACT',
 'THE GRAM  NYAYALAYAS ACT']

In [ ]:
def ideal_context(arr):
  count = 0
  text = ''
  final_arr = []

  for sen in arr:
    count += len(sen)
    text += sen + '. '
    if count < 300:
      continue
    else:
      final_arr.append(text)
      count = 0
      text = ''

  if len(text) != 0:
    final_arr.append(text)

  return final_arr

In [ ]:
train_file = []
i = 0
for doc in documents:
  title = titles[i].lower()
  ind = doc.find(title)
  final_text = doc[ind + len(title)+5:]
  removed_arr = final_text.split(".")
  removed_punc_arr = ideal_context(removed_arr)
  sentences = get_answer(removed_punc_arr)

  indexes = get_ans_ind(sentences)

  train_file.append(gen_que([],sentences,indexes))

  i+=1

Token indices sequence length is longer than the specified maximum sequence length for this model (631 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# train_file = []
# i = 0
# for doc in documents:
#   useful_content = False
#   title = titles[i]
#   for text in doc:
#     if title in text:
#       useful_content = True

#     if useful_content == True:
#       final_context = remove_puncts(text,string).strip()
#       split_text = final_context.split(".")

#       removed_punc_arr = []
#       for line in split_text:
#         removed_punc_arr.append(remove_puncts(line,string))

#       sentences = get_answer(removed_punc_arr)

#       indexes = get_ans_ind(sentences,final_context)

#       train_file.append(gen_que([],sentences,indexes))

#   i+=1

In [ ]:
len(train_file)

15

In [ ]:
final_file = []
for i in train_file:
  for j in i:
    final_file.append(j)

len(final_file)

1891

In [ ]:
#Prepare json file format

import json

# Specify the file path where you want to save the JSON data
file_path = "/content/acts.json"

# Open the file in write mode and dump the dictionary to it
with open(file_path, "w") as json_file:
    json.dump(final_file, json_file)